In [1]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split


In [2]:
def getBorderColor(img):
    
    border = np.asarray(img[0,:])
    border = np.concatenate((border, np.asarray(img[-1, :])))
    border = np.concatenate((border, np.asarray(img[:, 0])))
    border = np.concatenate((border, np.asarray(img[:, -1])))
    return np.bincount(border).argmax()


def pre_processing(img):
    _, img_binarized = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)    
    if(getBorderColor(img_binarized) != 0):
        img_binarized = cv2.bitwise_not(img_binarized) 
    return img_binarized



def showImage(img):
    plt.imshow(img, cmap='gray')
    plt.show()    



def getFeaturesHOG(img):
   img = cv2.resize(img, (256, 128))
   cell_size = (32, 32)  # h x w in pixels
   block_size = (2, 2)  # h x w in cells
   nbins = 9  # number of orientation bins
   # winSize is the size of the image cropped to an multiple of the cell size
   hog = cv2.HOGDescriptor(_winSize=(img.shape[1] // cell_size[1] * cell_size[1],
                                    img.shape[0] // cell_size[0] * cell_size[0]),
                            _blockSize=(block_size[1] * cell_size[1],
                                        block_size[0] * cell_size[0]),
                            _blockStride=(cell_size[1], cell_size[0]),
                            _cellSize=(cell_size[1], cell_size[0]),
                            _nbins=nbins)

   hog_feats = hog.compute(img)
   return hog_feats.flatten()     

# Read Data, Split Training, Validation, Test

In [3]:
data_set = []
Y = []
for i in range(1, 10):
    for filename in os.listdir("ACdata_base/" + str(i)):
        img = cv2.imread(os.path.join("ACdata_base/" + str(i),filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            data_set.append(pre_processing(img))
            Y.append(i)
            
X_train, X_testValid, Y_train, Y_testValid = train_test_split(data_set, Y, test_size=0.4, random_state=42)
X_validation, X_test, Y_validation, Y_test = train_test_split(X_testValid, Y_testValid, test_size=0.5, random_state=42)            

In [4]:

# for i in range(len(test_set)):
#     for j in range(len(test_set[i])):
#         showImage(test_set[i][j])

In [5]:
def getFeatures(images):
   x_features = []
   for i in range(len(images)):
        x_features.append(getFeaturesHOG(images[i]))
   return np.asarray(x_features)

In [6]:
train_features = getFeatures(X_train)
validation_features = getFeatures(X_validation)
test_features = getFeatures(X_test)
print(train_features.shape)
print(len(Y_train))


(1011, 756)
1011


In [12]:
#print(train_features[0])
#print(train_features[1])

In [7]:
import torch

In [40]:
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = train_features.shape[0], train_features.shape[1], 512, 9

# Create random Tensors to hold inputs and outputs
x = torch.tensor(train_features)
y = torch.tensor(Y_train,dtype=torch.float64)
y = y-1


In [42]:
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
    torch.nn.Softmax()
)

# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.
loss_fn = torch.nn.CrossEntropyLoss()

learning_rate = 1e-4
for t in range(2000):
    
    y_pred = torch.argmax(model(x), dim=1)
    
    print(y_pred)
    print(y)
    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero the gradients before running the backward pass.
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

tensor([2, 2, 2,  ..., 2, 2, 2])
tensor([7., 7., 3.,  ..., 4., 7., 6.], dtype=torch.float64)


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)